In [1]:
/* 18-7 */
Use AdventureWorks2014;
GO
CREATE TABLE dbo.DimProductSalesperson
(DimProductSalespersonID int IDENTITY(1,1) NOT NULL PRIMARY KEY,
ProductCD char(10) NOT NULL,
CompanyNBR int NOT NULL,
SalespersonNBR int NOT NULL );
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.011

In [2]:
/* Create staging table */

Use AdventureWorks2014;
GO
CREATE TABLE dbo.Staging_PRODSLSP ( ProductCD char(10) NOT NULL,
CompanyNBR int NOT NULL,
SalespersonNBR int NOT NULL );
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.011

In [3]:
/* Next, I’ll insert two rows into the staging table. */
Use AdventureWorks2014;
GO
INSERT dbo.Staging_PRODSLSP (ProductCD, CompanyNBR, SalespersonNBR) 
      VALUES ('2391A23904', 1, 24);
INSERT dbo.Staging_PRODSLSP (ProductCD, CompanyNBR, SalespersonNBR) 
      VALUES ('X129483203', 1, 34);
GO

Commands completed successfully.

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.009

In [4]:
/* insert into prod table */
Use AdventureWorks2014;
GO
INSERT Into dbo.DimProductSalesperson (ProductCD, CompanyNBR, SalespersonNBR) 
      SELECT s.ProductCD, s.CompanyNBR, s.SalespersonNBR 
            FROM dbo.Staging_PRODSLSP s 
            LEFT OUTER JOIN dbo.DimProductSalesperson d 
                  ON s.ProductCD = d.ProductCD 
                  AND s.CompanyNBR = d.CompanyNBR 
                  AND s.SalespersonNBR = d.SalespersonNBR 
      WHERE d.DimProductSalespersonID IS NULL;
GO

Commands completed successfully.

(2 rows affected)

Total execution time: 00:00:00.039

In [5]:
/* now create a udf */
Use AdventureWorks2014;
GO
CREATE FUNCTION dbo.udf_GET_Check_NK_DimProductSalesperson (@ProductCD char(10), @CompanyNBR int, @SalespersonNBR int ) 
RETURNS bit 
AS 
BEGIN
DECLARE @Exists bit
IF EXISTS (SELECT DimProductSalespersonID 
            FROM dbo.DimProductSalesperson 
            WHERE @ProductCD = @ProductCD 
            AND @CompanyNBR = @CompanyNBR 
            AND @SalespersonNBR = @SalespersonNBR) 
BEGIN
      SET @Exists = 1;
END 
ELSE 
BEGIN
      SET @Exists = 0;
END
RETURN @Exists 
END
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.008

In [6]:
/* rewritten insert statement */
Use AdventureWorks2014;
GO
INSERT INTO dbo.DimProductSalesperson(ProductCD, CompanyNBR, SalespersonNBR)
      SELECT ProductCD, CompanyNBR, SalespersonNBR
      FROM dbo.Staging_PRODSLSP
      WHERE dbo.udf_GET_Check_NK_DimProductSalesperson
       (ProductCD, CompanyNBR, SalespersonNBR) = 0;
GO

Commands completed successfully.

(0 rows affected)

Total execution time: 00:00:00.023